# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
df = pd.read_csv('C:/Users/ASUS/OneDrive/Desktop/ironhacklab/week_seven/day_1/lab_1/lab-chains-in-langchain/data/Data.csv')

In [12]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
#!pip install langchain_community

In [13]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [27]:
prompt = ChatPromptTemplate.from_template( "give me way the peppoal like this product{product} "
 
)

In [28]:

chain = LLMChain(llm=llm, prompt=prompt)

In [29]:
product = 'Queen Size Sheet Set'	
chain.run(product)

"1. Comfortable and soft material - People appreciate the feeling of luxurious comfort when they slip into bed with the Queen Size Sheet Set. The softness and smoothness of the sheets leave them feeling relaxed and help them have a restful night's sleep.\n\n2. Stylish and attractive design - The Queen Size Sheet Set comes in a variety of colors and patterns that appeal to different tastes and preferences. People enjoy being able to choose a design that complements their bedroom décor and adds a touch of style to their bedding.\n\n3. Easy to care for - The Queen Size Sheet Set is easy to care for and maintain, making it a convenient option for busy individuals. The sheets are durable and can withstand regular washing and drying without losing their quality or color.\n\n4. Perfect fit for their mattress - People appreciate that the Queen Size Sheet Set is designed to fit their Queen size mattress perfectly, ensuring a smooth and snug fit that stays in place throughout the night.\n\n5. Va

## SimpleSequentialChain

In [23]:
from langchain.chains import SimpleSequentialChain

In [30]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
   'tell me what this product commen problem of this {product} '
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [34]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'give a solution for this problem{chain_one}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [35]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [36]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Some common problems that consumers may face with a Queen Size Sheet Set include:
- Fading of colors after multiple washes
- Shrinking of the sheets after washing and drying
- Pilling of the fabric causing rough patches
- Elastic on fitted sheets losing its stretch and slipping off the mattress
- Poor quality stitching leading to tears or fraying of the sheets
- Unpleasant odor from the sheets that persists even after washing
One solution to prevent these common problems with a Queen Size Sheet Set is to invest in high-quality sheets made from durable materials such as 100% cotton or a cotton blend. Look for sheets with a high thread count for softer, more durable fabric.

To prevent fading of colors, wash the sheets in cold water with a gentle detergent and avoid using bleach or harsh chemicals. To prevent shrinking, follow the manufacturer's care instructions and avoid using high heat when drying the sheets.

To prevent pilling, wash th

"One solution to prevent these common problems with a Queen Size Sheet Set is to invest in high-quality sheets made from durable materials such as 100% cotton or a cotton blend. Look for sheets with a high thread count for softer, more durable fabric.\n\nTo prevent fading of colors, wash the sheets in cold water with a gentle detergent and avoid using bleach or harsh chemicals. To prevent shrinking, follow the manufacturer's care instructions and avoid using high heat when drying the sheets.\n\nTo prevent pilling, wash the sheets inside out and use a fabric softener or dryer balls to reduce friction. To prevent the elastic on fitted sheets from losing its stretch, look for sheets with deep pockets that can accommodate thick mattresses and consider using sheet straps or grippers to keep the sheets in place.\n\nTo prevent poor quality stitching, inspect the sheets for any loose threads or weak seams before purchasing. Opt for double-stitched or reinforced seams for added durability. To e

**Repeat the above twice for different products**

## SequentialChain

In [50]:
from langchain.chains import SequentialChain

In [56]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
"  translate this review from english to spain{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translation_review" #Give a name to your output
                    )


In [57]:
second_prompt = ChatPromptTemplate.from_template(
    "summarize a review to be short {translation_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summarize_review" #give a name to this output
                    )


In [58]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "translate this review{summarize_review} to from spain to arabic"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translate"
                      )


In [60]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "make this review more sempel {translate} "
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="message"
                     )


In [62]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translation_review", "summarize_review", "translate"],
    verbose=True
)

In [63]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translation_review': 'Encuentro el sabor mediocre. La espuma no dura, es extraño. Compro los mismos en el comercio y el sabor es mucho mejor... ¿Lote antiguo o falsificación!?',
 'summarize_review': 'El sabor es mediocre y la espuma no dura, me parece extraño. He comprado el mismo producto en tiendas y el sabor es mucho mejor. ¿Podría ser un lote antiguo o una falsificación?',
 'translate': 'الطعم متوسط \u200b\u200bوالرغوة لا تدوم ، يبدو غريبًا بالنسبة لي. لقد اشتريت نفس المنتج في المتاجر والطعم أفضل بكثير. هل يمكن أن يكون دُفعة قديمة أو تزوير؟'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [64]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [65]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [66]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [67]:
llm = ChatOpenAI(temperature=0)

In [68]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [69]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [70]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [71]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [72]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [73]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [74]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [75]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation of cells to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential fo

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [77]:
chain.run("What is 2**5")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2^5'}
> Finished chain.


'2^5 is equal to 32.'